In [2]:
import getkey
import json
import pandas as pd
import folium
import glob

In [2]:
# 道路壅塞水準講的只是各個level代表的意思（從小到大代表越壅塞）所以用不到
# 先取得省道即時發布路況，會有道路壅塞程度以及sectionID
# 再取得省道發布路段，會有sectionID, linkID
# 最後取得車輛偵測器，會有linkID, 經緯度就可以做成熱點圖
# 資料前處理將這些都抓下來，然後處理成可以顯示的資料
# 

In [3]:
# 要跟學長說的事：
# 省道、高速公路及縣市路段的資料為不同格式，但擁有的變數相同
# TDX資料好像有更新，找不到道路的始末位置


# 抓取省道資料

In [3]:
#抓高速公路即時發布路況
getkey.getjson("https://tdx.transportdata.tw/api/basic/v2/Road/Traffic/Live/Highway?%24format=JSON", "Congestion")

In [4]:
with open('data/Congestion.json', 'r', encoding='utf-8') as f:
    Congestion = json.load(f)

In [5]:
#抓高速公路發布路段
getkey.getjson("https://tdx.transportdata.tw/api/basic/v2/Road/Traffic/Section/Highway?%24format=JSON", "Section")

In [6]:
with open('data/Section.json', 'r', encoding='utf-8') as f:
    Section = json.load(f)

In [7]:
#抓車輛偵測器
getkey.getjson("https://tdx.transportdata.tw/api/basic/v2/Road/Traffic/VD/Highway?%24format=JSON", "VD")

In [8]:
with open('data/VD.json', 'r', encoding='utf-8') as f:
    VD = json.load(f)

In [9]:
pd_Congestion = pd.DataFrame(Congestion)
LiveTraffics = pd.DataFrame(pd_Congestion['LiveTraffics'])
LiveTraffics['SectionID'] = LiveTraffics['LiveTraffics'].apply(lambda x: x['SectionID'])
LiveTraffics['CongestionLevel'] = LiveTraffics['LiveTraffics'].apply(lambda x: x['CongestionLevel'])
# LiveTraffics
FreewayCongestion = LiveTraffics.drop(columns='LiveTraffics')
FreewayCongestion




,SectionID,CongestionLevel
0,1006100000210Q,1
1,1006100000220F,1
2,1006100001200F,1
3,1006100001300F,1
4,1006100001400F,1
...,...,...
6793,RF65001004K500001K800,1
6794,RF65001005K300004K500,1
6795,RF65001007K500005K300,1
6796,RF65001010K700007K500,1


In [10]:
pd_Section = pd.DataFrame(Section)
Sections = pd.DataFrame(pd_Section['Sections'])
Sections['SectionID'] = Sections['Sections'].apply(lambda x : x['SectionID'])
Sections['LinkIDs'] = Sections['Sections'].apply(lambda x : x['LinkIDs'])
Sections = Sections.drop(columns='Sections')
Sections = Sections.explode('LinkIDs')
Sections['LinkID'] = Sections['LinkIDs'].apply(lambda x: x['LinkID'])
FreewaySection = Sections.drop(columns = 'LinkIDs')
FreewaySection


,SectionID,LinkID
0,BB10020000K000000K150,3001020000000B
1,BB10021000K150000K000,3001020100000B
2,BB10020000K150000K300,3001020000013B
3,BB10021000K300000K150,3001020100013B
4,BB10020000K300000K780,3001020000026B
...,...,...
6676,3000900100926F,3000900100926F
6677,3000900000968F,3000900000968F
6678,3000900100968F,3000900100968F
6679,3000900000992F,3000900000992F


In [31]:
# pd_Section = pd.DataFrame(Section)
# Sections = pd.DataFrame(pd_Section['Sections'])
# Sections['SectionID'] = Sections['Sections'].apply(lambda x: x['SectionID'])
# # Sections['LinkIDs'] = Sections['Sections'].apply(lambda x : x['LinkIDs'])
# # Sections['Sections'] = Sections['Sections'].apply(lambda x: x['SectionID'])
# # Sections['LinkIDs'] = Sections['Sections'].apply(lambda x: [item['LinkID'] for item in x['LinkIDs']])
# Sections


In [11]:
pd_VD = pd.DataFrame(VD)
VDs = pd.DataFrame(pd_VD, columns=['VDs'])
VDs['VDID'] = VDs['VDs'].apply(lambda x : x['VDID'])
VDs['DetectionLinks'] = VDs['VDs'].apply(lambda x : x['DetectionLinks'])
VDs['PositionLat'] = VDs['VDs'].apply(lambda x : x['PositionLat'])
VDs['PositionLon'] = VDs['VDs'].apply(lambda x : x['PositionLon'])
FreewayVD = VDs.drop(columns='VDs')
# FreewayVD
FreewayVD = FreewayVD.explode('DetectionLinks')
FreewayVD['LinkID'] = FreewayVD['DetectionLinks'].apply(lambda x: x['LinkID'])
FreewayVD = FreewayVD.drop(columns='DetectionLinks')
FreewayVD





,VDID,PositionLat,PositionLon,LinkID
0,VD-33-0090-380-001,22.774262,121.083714,3000900136433V
0,VD-33-0090-380-001,22.774262,121.083714,3000900036433V
1,VD-12-0640-011-006,25.080916,121.448471,1006401005060F
2,VD-23-0140-007-001,24.058689,120.618494,3001400100695N
2,VD-23-0140-007-001,24.058689,120.618494,3001400000700N
...,...,...,...,...
1606,VD-T74-E-22-O-NS-1-南向出口-松竹路,24.183638,120.721740,1007401013000B
1607,VD-T84-E-17.895-M-RS,23.195824,120.268845,1008400001700D
1608,VD-T84-E-13-I-SE-21-Loop,23.217536,120.241909,1008401003080D
1609,VD-T78-W-22-I-EW-1-西向入口,23.691969,120.384449,1007801104030P


In [12]:
SectionCongestion = pd.merge(FreewayCongestion,FreewaySection, how='inner', on='SectionID')
FreewayData = pd.merge(FreewayVD,SectionCongestion, how='inner', on='LinkID')
FreewayData = FreewayData.drop(columns='VDID')
FreewayData = FreewayData.drop(columns='LinkID')
FreewayData = FreewayData.drop(columns='SectionID')
FreewayData


,PositionLat,PositionLon,CongestionLevel
0,22.774262,121.083714,1
1,22.774262,121.083714,1
2,24.058689,120.618494,1
3,24.058689,120.618494,1
4,25.040183,121.243227,1
...,...,...,...
1380,23.658170,121.453950,1
1381,23.658170,121.453950,1
1382,24.085463,120.596841,1
1383,24.085642,120.597176,1


In [15]:
from folium.plugins import HeatMap

FreewayData['PositionLat'] = FreewayData['PositionLat'].astype(float)
FreewayData['PositionLon'] = FreewayData['PositionLon'].astype(float)
FreewayData['CongestionLevel'] = FreewayData['CongestionLevel'].astype(float)

hm = folium.Map(location = [23.5,121],zoom_start=8)

heat_data = [[row['PositionLat'], row['PositionLon'], row['CongestionLevel']] for index, row in FreewayData.iterrows()]

HeatMap(heat_data, radius = 10).add_to(hm)

hm